In [44]:
# Étape 1 - Connexion à la base de données et récupération d’une table existante (accidents_cleaned)
import sys
sys.path.append("..")  # remonte d'un niveau vers le dossier parent
from utils.utils import connect_to_db, insert_data_to_db, mapping_colonnes, fetch_data_from_db
engine = connect_to_db()
df = fetch_data_from_db(table_name='accidents_cleaned', engine=engine, schema='silver')
df

✅ Connexion à la base PostgreSQL réussie !
OK: "silver"."accidents_cleaned"


,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,year_georef,com_name,dep_code,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,code_com
0,201900031048,2019-11-09 13:30:00+00:00,Caudan,2019,11,9,14:30,Plein jour,En agglomération,1,...,2019,Caudan,56,Morbihan,200042174,CA Lorient Agglomération,53,Bretagne,Caudan,56036
1,201900033371,2019-06-21 16:05:00+00:00,Montereau-fault-yonne,2019,6,21,18:05,Plein jour,En agglomération,1,...,2019,Montereau-Fault-Yonne,77,Seine-et-Marne,247700107,CC Pays de Montereau,11,Île-de-France,Montereau-Fault-Yonne,77305
2,201900035071,2019-06-23 01:20:00+00:00,Gigean,2019,6,23,03:20,Nuit sans éclairage public,Hors agglomération,1,...,2019,Gigean,34,Hérault,200066355,CA Sète Agglopôle Méditerranée,76,Occitanie,Gigean,34113
3,201400032968,2014-11-01 09:30:00+00:00,Tarbes,2014,11,1,10:30,Plein jour,En agglomération,1,...,2015,Tarbes,65,Hautes-Pyrénées,246500565,CA du Grand Tarbes,73,Midi-Pyrénées,Tarbes,65440
4,201600013686,2016-06-26 11:40:00+00:00,La Ciotat,2016,6,26,13:40,Plein jour,En agglomération,1,...,2016,La Ciotat,13,Bouches-du-Rhône,200054807,Métropole d'Aix-Marseille-Provence,93,Provence-Alpes-Côte d'Azur,La Ciotat,13028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201200004488,2012-06-18 20:30:00+00:00,Montech,2012,6,18,22:30,Nuit sans éclairage public,Hors agglomération,1,...,2015,Montech,82,Tarn-et-Garonne,248200180,CC Garonne et Canal,73,Midi-Pyrénées,Montech,82125
475907,201900026968,2019-07-29 14:20:00+00:00,Bannieres,2019,7,29,16:20,Plein jour,Hors agglomération,1,...,2019,Bannières,81,Tarn,200034023,CC Tarn-Agout,76,Occitanie,Bannières,81022
475908,201900028113,2019-09-07 15:25:00+00:00,Lorris,2019,9,7,17:25,Plein jour,Hors agglomération,1,...,2019,Lorris,45,Loiret,200067676,CC Canaux et Forêts en Gâtinais,24,Centre-Val de Loire,Lorris,45187
475909,201900028302,2019-06-28 07:45:00+00:00,Volx,2019,6,28,09:45,Plein jour,Hors agglomération,1,...,2019,Volx,04,Alpes-de-Haute-Provence,200034700,CA Durance-Lubéron-Verdon Agglomération,93,Provence-Alpes-Côte d'Azur,Volx,4245


In [45]:
# Définition de la fonction insert_data_to_db() : version robuste qui accepte le paramètre schema, gère les erreurs et affiche un message clair après insertion
def insert_data_to_db(df, table_name, engine, schema=None):
    """
    Insère un DataFrame dans une table PostgreSQL via SQLAlchemy.
    
    :param df: DataFrame à insérer
    :param table_name: Nom de la table
    :param engine: Objet SQLAlchemy Engine
    :param schema: Schéma PostgreSQL (par défaut None)
    """
    try:
        df.to_sql(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists='replace',  # ou 'append' si tu veux ajouter sans écraser
            index=False
        )
        print(f"✅ Insertion réussie dans {schema}.{table_name} ({len(df)} lignes)")
    except Exception as e:
        print(f"❌ Erreur lors de l'insertion dans {schema}.{table_name} : {e}")

In [46]:
# Étape 2 - Extraction des colonnes pour créer deux DataFrames (df_personnes et df_departement) à partir du DataFrame source.
colonnes_personnes = [
    'num_acc', 'num_veh', 'an_nais', 'sexe', 'actp', 'grav',
    'locp', 'catu', 'etatp', 'occutc'
]


colonnes_departement = [
    'dep_code', 'dep_name', 'reg_code', 'insee', 'dep'
]

df_personnes = df[colonnes_personnes].copy()
df_departement = df[colonnes_departement].copy()

df_personnes.head(100)
df_departement.head()

,dep_code,dep_name,reg_code,insee,dep
0,56,Morbihan,53,56036,56
1,77,Seine-et-Marne,11,77305,77
2,34,Hérault,76,34113,34
3,65,Hautes-Pyrénées,73,65440,65
4,13,Bouches-du-Rhône,93,13028,13


In [47]:
# Étape 3 - Nettoyage simple dans pandas
# Identification des doublons
print(df_personnes.duplicated().sum())
print(df_departement.duplicated().sum())

0
436630


In [48]:
# Limiter le risque de suppression abusive :
# Nombre de lignes ayant des clés manquantes (Vérifier combien de lignes seraient supprimées avant d’agir)
print(df_personnes[['num_acc', 'num_veh']].isna().sum())
print(df_personnes[df_personnes[['num_acc', 'num_veh']].isna().any(axis=1)])

num_acc    0
num_veh    0
dtype: int64
Empty DataFrame
Columns: [num_acc, num_veh, an_nais, sexe, actp, grav, locp, catu, etatp, occutc]
Index: []


In [49]:
# Supprimer doublons sur la clé composite
df_personnes = df_personnes.drop_duplicates(subset=['num_acc', 'num_veh'])
df_departement = df_departement.drop_duplicates(subset=['dep_code', 'reg_code'])

# Supprimer lignes avec clés manquantes
df_personnes = df_personnes.dropna(subset=['num_acc', 'num_veh'])
df_departement = df_departement.dropna(subset=['dep_code', 'reg_code'])

In [50]:
# Identification des valeurs manquantes
# L'indicateur le plus pertinent n'est pas forcément le nb absolu de valeurs manquantes mais la proportion de valeurs manquantes par colonne.
# Vérification des valeurs manquantes dans df_personne (méthode ISNA)
# Vérification du nombre de lignes NaN, vides ou avec des valeurs fréquentes dans la colonne 'an_nais' de df_personnes.
# Vérification des valeurs manquantes dans df_personne (méthode ISNULL)
print("Valeurs manquantes dans df_personnes :")
print(df_personnes.isnull().sum())

# Vérification des valeurs manquantes dans df_departement
print("\nValeurs manquantes dans df_departement :")
print(df_departement.isnull().sum())

Valeurs manquantes dans df_personnes :
num_acc         0
num_veh         0
an_nais    385713
sexe            0
actp        21827
grav            0
locp       374069
catu            0
etatp      343171
occutc     469896
dtype: int64

Valeurs manquantes dans df_departement :
dep_code    0
dep_name    0
reg_code    0
insee       0
dep         0
dtype: int64


✅ Pour df_personnes :

num_acc : 0 → aucune valeur manquante.
num_veh : 0 → aucune valeur manquante.
an_nais : 385 713 → énorme nombre de valeurs manquantes (année de naissance).
sexe : 0 → complet.
actp : 21 827 → certaines valeurs manquantes (type d’action ?).
grav : 0 → complet.
locp : 374 069 → très nombreuses valeurs manquantes (localisation ?).
catu : 0 → complet.
etap : 343 171 → beaucoup de manquants.
occutc : 469 896 → énorme nombre de manquants.

👉 Synthèse des colonnes :

- an_nais, locp, etap, occutc        ==> très incomplètes.
- num_acc, num_veh, sexe, grav, catu ==> fiables (aucun manquant).

In [51]:
# Analyse —> colonne 'année_de_naissance' (NaN : 385 713 lignes)
# Proportion des valeurs manquantes 
missing_fraction = df_personnes['an_nais'].isna().mean()
print(f"Fraction des valeurs manquantes dans 'an_nais': {missing_fraction:.2%}")

Fraction des valeurs manquantes dans 'an_nais': 81.05%


In [52]:
# Analyse —> colonne 'localisation_du_piéton' (NaN : 374 069 lignes)
# Proportion des valeurs manquantes 
missing_fraction = df_personnes['locp'].isna().mean()
print(f"Fraction des valeurs manquantes dans 'locp': {missing_fraction:.2%}")

Fraction des valeurs manquantes dans 'locp': 78.60%


In [53]:
# Analyse —> colonne 'piéton_seul_ou_non' (NaN : 343 171 lignes)
# Proportion des valeurs manquantes 
missing_fraction = df_personnes['etatp'].isna().mean()
print(f"Fraction des valeurs manquantes dans 'etatp': {missing_fraction:.2%}")

Fraction des valeurs manquantes dans 'etatp': 72.11%


In [54]:
# Analyse —> colonne 'Nb-occupants' (NaN : 469 896 lignes)
# Proportion des valeurs manquantes 
missing_fraction = df_personnes['occutc'].isna().mean()
print(f"Fraction des valeurs manquantes dans 'occutc': {missing_fraction:.2%}")

Fraction des valeurs manquantes dans 'occutc': 98.74%


In [55]:
# Gestion de valeurs manquantes  
# df_personnes = df_personnes.dropna(subset=['num_acc', 'num_veh'])  # Critique pour clé étrangère
# df_departement = df_departement.dropna(subset=['dep_code', 'dep_name', 'reg_code'])

# Remplacement des valeurs manquantes dans df_personnes
df_personnes['an_nais'] = df_personnes['an_nais'].fillna('inconnu')  # évite des suppressions massives de données tout en indiquant clairement où les données sont manquantes.
df_personnes['locp'] = df_personnes['locp'].fillna('inconnu')
df_personnes['etatp'] = df_personnes['etatp'].fillna('inconnu')
df_personnes['occutc'] = df_personnes['occutc'].fillna('inconnu')
df_personnes['actp'] = df_personnes['actp'].fillna('inconnu')

In [ ]:
# Étape 4 - Traitement des colonnes avec multi-valeurs (séparées par des virgules)
# num_veh | an_nais | sexe | actp | grav | etatp | occutc
# A) Comparer pour chaque ligne le nombre de valeurs (après séparation par virgule) dans chaque colonne concernée, 
# afin de repérer les décalages ou anomalies avant d’exploser et structurer le DataFrame
# Application aux colonnes: 'num_veh', 'an_nais', 'sexe', 'actp', 'grav', 'occutc'

'''import pandas as pd

cols_multi = ['num_veh', 'an_nais', 'sexe', 'actp', 'grav', 'etatp', 'occutc']
df_personnes['num_veh'] = df_personnes['num_veh'].astype(str)
df_personnes['an_nais'] = df_personnes['an_nais'].astype(str)

def count_split(row, col):
    return len(row[col].split(',')) if pd.notna(row[col]) and row[col] != '' else 0

# Pour chaque colonne, crée une nouvelle colonne suffixée _count indiquant le nombre de valeurs par ligne
for col in cols_multi:
    df_personnes[col + '_count'] = df_personnes.apply(lambda row: count_split(row, col), axis=1)

# Pour visualiser les différences sur les premières lignes
print(df_personnes[[col + '_count' for col in cols_multi]].head())

# B) Exploser les colonnes multi-valeurs pour structurer le DataFrame
# Bloc 1 : Stratégie d’alignement entre catu et num_veh
# Ce bloc crée deux colonnes alignées : catu_list et num_veh_aligned

def align_num_veh(row):
    catu_list = row['catu'].split(',') if pd.notna(row['catu']) else []
    num_veh_list = row['num_veh'].split(',') if pd.notna(row['num_veh']) else []
    num_veh_aligned = []
    j = 0
    for cat in catu_list:
        if cat.strip().lower() in ['passager', 'piéton', 'piétonne', 'passagère']:
            num_veh_aligned.append('')  # pas de véhicule
        else:
            if j < len(num_veh_list):
                num_veh_aligned.append(num_veh_list[j])
                j += 1
            else:
                num_veh_aligned.append('')
    return pd.Series({'catu_list': catu_list, 'num_veh_aligned': num_veh_aligned})

df_personnes[['catu_list', 'num_veh_aligned']] = df_personnes.apply(align_num_veh, axis=1) '''

   num_veh_count  an_nais_count  sexe_count  actp_count  grav_count  \
0              1              1           2           2           2   
1              1              1           2           2           2   
2              1              1           1           1           1   
3              2              1           2           2           2   
4              2              1           2           2           2   

   etatp_count  occutc_count  
0            2             1  
1            2             1  
2            1             1  
3            1             1  
4            1             1  


In [ ]:
# Bloc 2 : Explosion de toutes les colonnes multivaluées et conservation de locp
# Ce bloc prépare les colonnes à exploser, les convertit en listes, puis explose le DataFrame :
'''cols_multi = ['an_nais', 'sexe', 'actp', 'grav', 'etatp', 'occutc']

for col in cols_multi:
    df_personnes[col + '_list'] = df_personnes[col].apply(lambda x: str(x).split(',') if pd.notna(x) else [''])

# Réunir toutes les colonnes à exploser
explode_cols = ['catu_list', 'num_veh_aligned'] + [col + '_list' for col in cols_multi]

df_personnes_exploded = df_personnes.explode(explode_cols, ignore_index=True)

# locp est inchangée, répliquée sur chaque personne
print(df_personnes_exploded[['num_acc', 'num_veh_aligned', 'an_nais_list', 'sexe_list', 'actp_list', 'grav_list', 'etatp_list', 'occutc_list', 'catu_list', 'locp']].head())'''


"cols_multi = ['an_nais', 'sexe', 'actp', 'grav', 'etatp', 'occutc']\n\nfor col in cols_multi:\n    df_personnes[col + '_list'] = df_personnes[col].apply(lambda x: str(x).split(',') if pd.notna(x) else [''])\n\n# Réunir toutes les colonnes à exploser\nexplode_cols = ['catu_list', 'num_veh_aligned'] + [col + '_list' for col in cols_multi]\n\ndf_personnes_exploded = df_personnes.explode(explode_cols, ignore_index=True)\n\n# locp est inchangée, répliquée sur chaque personne\nprint(df_personnes_exploded[['num_acc', 'num_veh_aligned', 'an_nais_list', 'sexe_list', 'actp_list', 'grav_list', 'etatp_list', 'occutc_list', 'catu_list', 'locp']].head())"

In [65]:
# 2.	Charger et lire le fichier .sql - Exécuter le script SQL dans PostgreSQL 
import sqlalchemy
from sqlalchemy import text
import os

base_dir = os.getcwd()               # si on est dans un notebook


sql_file_path = os.path.join(base_dir, "..", "stockage", "table_accident.sql")
sql_file_path = os.path.normpath(sql_file_path)  # normalisation du chemin
print(f"Chemin utilisé : {sql_file_path}")


if not os.path.exists(sql_file_path):
    raise FileNotFoundError(f"Le fichier SQL n'existe pas : {sql_file_path}")


with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_script = file.read()


print("⚙️ Création des tables dans le schéma SILVER...")
with engine.begin() as conn:
    # On indique le schéma 'silver' pour la session
    conn.execute(text("SET search_path TO silver, public;"))
    conn.execute(text(sql_script))

print("✅ Toutes les tables ont été créées avec succès dans le schéma SILVER !")

Chemin utilisé : c:\Users\utilisateur\OneDrive - Campus E.S.P.R.I.T. Industries\INFORMATIQUE-2024\SIMPLON 2025-2026\Alternance 2025-27\Brief_17-oct25\brief_1_accidents\stockage\table_accident.sql
⚙️ Création des tables dans le schéma SILVER...
✅ Toutes les tables ont été créées avec succès dans le schéma SILVER !


In [66]:
# Étape 5 - Définition des objets SQLAlchemy 
# Définition des tables avec leurs colonnes pour l’UPSERT.

from sqlalchemy import MetaData, Table, Column, String, Integer
from sqlalchemy.dialects.postgresql import insert

metadata = MetaData(schema="silver")

departement = Table(
    "departement", metadata,
    Column("dep_code", String, primary_key=True),
    Column("dep_name", String),
    Column("reg_code", String, primary_key=True),
    Column("dep", String),
    Column("insee", String)
)

personnes = Table(
    "personnes", metadata,
    Column("num_acc", String, primary_key=True),
    Column("num_veh", String, primary_key=True),
    Column("an_nais", Integer),
    Column("sexe", String),
    Column("locp", String),
    Column("catu", String),
    Column("etatp", String),
    Column("grav", String),
    Column("occutc", String)
)

In [67]:
# Fonction UPSERT avec logs
# Cette fonction gère les PK composites et affiche des logs pour suivre les opérations.
def upsert_dataframe(df, table, engine, conflict_cols):
    df = df.dropna(how="all")  # Nettoyage des lignes vides
    with engine.begin() as conn:
        for i, (_, row) in enumerate(df.iterrows(), start=1):
            stmt = insert(table).values(**row.to_dict())
            update_dict = {col: row[col] for col in df.columns if col not in conflict_cols}
            stmt = stmt.on_conflict_do_update(
                index_elements=conflict_cols,
                set_=update_dict
            )
            conn.execute(stmt)
            print(f"[UPSERT] Ligne {i}/{len(df)} insérée ou mise à jour dans {table.name}")

In [68]:
# Étape 7 - Insertion des données dans les DataFrames avec UPSERT

# Table departement
df_dep = fetch_data_from_db(table_name='departement', engine=engine, schema='silver')
df_pers = fetch_data_from_db(table_name='personnes', engine=engine, schema='silver')

OK: "silver"."departement"
OK: "silver"."personnes"


In [ ]:

# UPSERT vers silver
'''upsert_dataframe(df_dep, departement, engine, conflict_cols=["dep_code", "reg_code"])
upsert_dataframe(df_pers, personnes, engine, conflict_cols=["num_acc", "num_veh"]) '''


'upsert_dataframe(df_dep, departement, engine, conflict_cols=["dep_code", "reg_code"])\nupsert_dataframe(df_pers, personnes, engine, conflict_cols=["num_acc", "num_veh"]) '